### Beat the BTB with vowpall wabbit

General instructions to reproduces the submission that by this time would get you at the 90th position. Find the code in the [repo](https://github.com/martinbel/kaggle-outbrain-click-prediction).

**1.** Run csv2vw.py to read the data into the right format. You'll need to change the path of each file for this

The output file has a namespace for each variable, in order to allow making interactions more flexible.                                        

**ad_id - display_id:**

|a ad_id |b display_id

**events:**

|u uuid|d document_id|p promoted|c country|s state|l localoty|w week|h hour|

**promoted:**

|x document_id |y campaign_id |z advertiser_id

**2.** `vwcommands.sh` has an example of the model I've used to get to 0.64144 in the public leaderboard, that by this time is at the 90th position.
Not great but a nice start with room for improvement.
The average loss for this model is: 0.437686

**3.** write_submission.R converts the output of vw into probabilities, and finally the submission format. 

### First let's clone the repo

In [ ]:
!git clone https://github.com/martinbel/kaggle-outbrain-click-prediction
cd kaggle-outbrain-click-prediction

One simple way to move forward is to copy the data into this folder. Another option is to reference them where you have them by changing the scripts. 
The `csv2vw.py` script reads the files: 
- clicks_train.csv
- events.csv 
- promoted_content.csv
joins them and converts them to vowpal wabbit's [input format](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format). 

In [ ]:
# Run 
python csv2vw.py

Once the file is in the correct format, that can be checked with [validator](http://hunch.net/~vw/validate.html) we can train a model. 
The following model is a one layer neural network, trained with SGD. There are several [options](https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments) that can be explored. See the [tutorials](https://github.com/JohnLangford/vowpal_wabbit/wiki/Tutorial) for more info. 

In [ ]:
### Train model - average loss 0.437686
vw train.w -f nn_250.vw -c --loss_function logistic -l 0.01 -b 28 --l2 1e-7 --ignore b --nn 250 --dropout --passes 2

# Inference
vw test.w -t -i nn_250.vw -p prob_nn_250.txt

The last comands generates predictions from the model and saves them in prob_nn_250.txt. We still need to convert these raw scores (they are not probabilities) into the submission format.

For this run the `write_submission.R` script that reads the prob_nn_250.txt, clicks_test.csv and writes the submission file: `btb_with_vw.csv`.